# Hybrid Monte Carlo

## Products And Exposures

In this notebook we demonstrate the setup of products and the calculation of exposures. In practice products are specified by actual dates (not model times). We use QuantLib to handle date arithmetics (calenders, day counts, etc.) in product specifications.

This notebook is structured along the following sections:

  1.  Setting up a cash flow leg in QuantLib
  2.  Constructing and inspecting a swap product
  3.  Calculating exposures with MC and AMC

We use a couple of standard packages for calculation and analysis

In [ ]:
import sys
sys.path.append('../')  # make python find our modules
import numpy as np
import pandas as pd
import plotly.express as px

The following auxilliary method lists the relevant members of an object. We use it to inspect the objects created. 

In [ ]:
def members(obj):
    return [f for f in dir(obj) if not f.startswith('_')] 

### Setting up a cash flow leg in QuantLib

For general details on QuantLib see e.g. https://www.quantlib.org/

Here we demonstrate the features required to set up simple cash flow legs.

QuantLib has a global evaluation date. This is the date from which time periods for term structures and models are calculated. We set evaluation date at inception and keep it fixed during the run of the session.

In [ ]:
import QuantLib as ql
today     = ql.Date(5,ql.October,2020)
ql.Settings.instance().evaluationDate = today

We want to set up a fixed leg and a standard floating leg. For multi-curve modelling we need to take into account the tenor basis. Roughly speaking, tenor basis is the difference between projection curve and discount curve. Consequently, to model tenor basis we need a projection curve and a discount curve.

In [ ]:
discYtsH = ql.YieldTermStructureHandle(
            ql.FlatForward(today,0.01,ql.Actual365Fixed()))
projYtsH = ql.YieldTermStructureHandle(
            ql.FlatForward(today,0.02,ql.Actual365Fixed()))

In this example we set discount curve flat at *1%* continuous componded zero rate and projection curve flat at *2%* continuous componded zero rate. A *ql.FlatForward()* object is equivalent to our *YieldCurve()* object. QuantLib's *ql.YieldTermStructureHandle()* is just a double-indirection for the underlying curve.

Euribor/Libor forward rates are modelled in QunatLib via indizes. We set up an *Euribor* index.

In [ ]:
index = ql.Euribor6M(projYtsH)

Cash flow dates are specified via *Schedule* objects. For details on how to construct QuantLib schedules see https://github.com/lballabio/QuantLib-SWIG/blob/master/SWIG/scheduler.i.

In [ ]:
# we set start in the future to avoid the need of index fixings
startDate  = ql.Date(12,ql.October,2020)
endDate    = ql.Date(12,ql.October,2030)
calendar   = ql.TARGET()
fixedTenor = ql.Period('1y')
floatTenor = ql.Period('6m')
fixedSchedule = ql.MakeSchedule(startDate,endDate,tenor=fixedTenor,calendar=calendar)
floatSchedule = ql.MakeSchedule(startDate,endDate,tenor=floatTenor,calendar=calendar)

A schedule behaves essentially like a list of dates.

In [ ]:
display(list(fixedSchedule))

Now we can setup a fixed leg. For details on how to setup cash flow legs with QuantLib see https://github.com/lballabio/QuantLib-SWIG/blob/master/SWIG/cashflows.i.

In [ ]:
couponDayCount = ql.Thirty360()
notional = 1.0
fixedRate = 0.02
fixedLeg = ql.FixedRateLeg(fixedSchedule,couponDayCount,[notional],[fixedRate])

Similarly, we can setup a floating rate leg.

In [ ]:
floatingLeg = ql.IborLeg([notional],floatSchedule,index)

Cash flow legs behave like lists of cash flows.

In [ ]:
display([ cf.amount() for cf in fixedLeg ])

We can calculate present value of the legs using the discount curve.

In [ ]:
display('FixedLeg npv:    %.4f' % ql.CashFlows_npv(fixedLeg,discYtsH,True))
display('FloatingLeg npv: %.4f' % ql.CashFlows_npv(floatingLeg,discYtsH,True))

### Constructing and inspecting a swap product

A *Product* object represents a financial instrument with one or several cash flows. Each cash flow is represented as a Monte Carlo payoff $V_i(T_i)$ that is paid at $T_i$.

The *Product* objects implement a function *cashflows(obsTime)*. This function calculates payoffs with observation time $t$ that calculate (or estimate)
$$
  V(t) = B(t) \mathbb{E} \left[
      \sum_{T_i>t} \frac{V_i(T_i)}{B(T_i)} \, | \, \cal{F}_t
      \right]
      = \sum_{T_i>t} \mathbb{E} \left[
      \frac{V_i(T_i)}{B(T_i)} \, | \, \cal{F}_t
      \right].
$$

If the payoffs $V_i$ are simple enough such that $\mathbb{E} \left[ V_i(T_i) / B(T_i) \, | \, \cal{F}_t \right]$ can be calculated in closed form then we use this analytic expression in the cash flow method. This is typically the case for linear products and Vanilla options.

For complex payoffs without analytical expression for $\mathbb{E} \left[ V_i(T_i) / B(T_i) \, | \, \cal{F}_t \right]$ we use AMC to estimate the conditional expectation.

We implement a *Swap* product that uses the discounted cash flows and forward Libor rates for the analytical payoff expessions.

A *Swap* product is represented by a list of fixed or Ibor legs. Moreover, we need to specify whether we receive (+1) or pay (-1) a leg. The calculation of tenor basis also requires the discount curve.

In [ ]:
from hybmc.products.Swap import Swap
swap = Swap([fixedLeg,floatingLeg],[1.0,-1.0],discYtsH)

We check the cash flow calculation at $t=9.0$, i.e. approximately 1y bevor swap maturity.

In [ ]:
cfs = swap.cashFlows(9.0)
display([str(cf) for cf in cfs])

We make the following observations:
  -  We have two remaining fixed leg payments. The first fixed leg payment is paid at $T=9.02$ and
     the second fixed leg payment is paid at $T=10.03$.
  -  Similarly, we have three remaining floating rate payments.
  -  Payoffs are discounted to time $t=9.0$, see *P_None(9.00,.)*.
  -  Payoffs are observed at $t=9.0$, see *@ 9.00*.

Also note that the first Libor cash flow has fixing time $8.52$ compared to $9.0$ for the other cash flows. This is correct, because at observation time $t=9.0$ the the Libor rate is alsready fixed but the coupon is not yet paid.

For the other Libor cash flows the actual Libor fixing is in the future (later than  $t=9.0$). However, we can calculate (using $T_i$-forward measure)
$$
  B(t) \cdot \mathbb{E} \left[ V_i(T_i) / B(T_i) \, | \, \cal{F}_t \right]
  =
  P(t,T_i) \cdot \mathbb{E}^{T_i} \left[ V_i(T_i) \, | \, \cal{F}_t \right].
$$
And for a Libor rate $\mathbb{E}^{T_i} \left[ L_i(T_i) \, | \, \cal{F}_t \right]$ becomes the forward Libor rate with observation time equal to $t$.


If we calculate cash flows at $t=0$ then we get the full list of product cash flows.

In [ ]:
cfs = swap.cashFlows(0.0)
display([str(cf) for cf in cfs])

We can use a deterministic model to calculate these payoffs.

In [ ]:
from hybmc.models.DeterministicModel import DcfModel
path = DcfModel(discYtsH).path()
amounts = np.array([ cf.discountedAt(path) for cf in cfs ])
display(amounts)

We can double-check the valuation against QuantLib's valuation of the fixed and float leg.

In [ ]:
amountsQl = np.array(
    [cf.amount() * discYtsH.discount(cf.date()) for cf in fixedLeg] +
    [-cf.amount() * discYtsH.discount(cf.date()) for cf in floatingLeg] )
display(amountsQl)

This looks good except the 12th floating rate cash flow (with index 11):

In [ ]:
display(str(cfs[21]))
display(cfs[21].discountedAt(path))
cf = list(floatingLeg)[11]
display(-cf.amount() * discYtsH.discount(cf.date()))

That is an interesting case. Differences are probably due to some slight date mismatch in accrual period versus fixing period.

### Calculating exposures with MC and AMC

Once the cash flow method is implemented for a product we can call it for a range of observation times. This gives a *time line* of payoffs.

In [ ]:
timeline = swap.timeLine([0.0, 3.0, 10.0])
for t in timeline:
    print('ObsTime: %.2f' % t)
    for p in timeline[t]:
        print(p)


For exposure valuation we need to set up a model and a MC simulation.

In [ ]:
from hybmc.models.HullWhiteModel import HullWhiteModel
from hybmc.simulations.McSimulation import McSimulation
model = HullWhiteModel(discYtsH,0.03,np.array([10.0]),np.array([0.0050]))
mcsim = McSimulation(model,np.linspace(0.0,10.0,41),2**10,314159265359,True)

Now we can calculate scenarios for the time line.

In [ ]:
times = np.linspace(0.0,10.0,41)
scens = swap.scenarios(times,mcsim)

For exposure simulation we are interested in the $\mathbb{E}[V(t)^+]$.

In [ ]:
epeDcf = np.average(np.maximum(scens,0.0),axis=0)

We plot the exposure profile.

In [ ]:

dfDcf = pd.DataFrame([ times, epeDcf ]).T
dfDcf.columns = ['times', 'epeDcf']
fig = px.line(dfDcf,x='times',y='epeDcf')
fig.show()

Alternatively (and as a proof of concept), we can also setup a swap time line using American Monte Carlo.

In [ ]:
from hybmc.products.Swap import AmcSwap
mcsim_training = McSimulation(model,np.linspace(0.0,10.0,41),2**10,2718281828,True)
swap = AmcSwap([fixedLeg,floatingLeg],[1.0,-1.0],mcsim_training,2,discYtsH)

In [ ]:
cfs = swap.cashFlows(9.0)
display([str(cf) for cf in cfs])

Here we see a single payoff per observation time. That payoff is a AMC regression payoff that references the actual swap payoffs at future pay times.

As regresssion variable we use a *co-terminal Libor rate*. That is a bit unusual, but it does the job.

Similarly, as with the analytic approach we calculate scenarion and expected (positive) exposures.

In [ ]:
scens = swap.scenarios(times,mcsim)
epeAmc = np.average(np.maximum(scens,0.0),axis=0)

In [ ]:
dfAmc = pd.DataFrame([ times, epeAmc ]).T
dfAmc.columns = ['times', 'epeAmc']
fig = px.line(dfAmc,x='times',y='epeAmc')
fig.show()

Finally, we compare profiles from analytic and AMC method.

In [ ]:
dfDcf.columns = ['times', 'epe']
dfDcf['type'] = 'Dcf'
dfAmc.columns = ['times', 'epe']
dfAmc['type'] = 'Amc'
df = pd.concat([dfDcf,dfAmc],axis=0)

In [ ]:
fig = px.line(df,x='times',y='epe', color='type')
fig.show()